# Green and Red Candle Project
## Coded by Victor Zheng & Supervised under Mohammed Hasan

In [136]:
## Progress
# 06-12-2021: Find RSI Crossover; Get StrikePrice; Remade function for beforeTime,afterTime
# 06-13-2021: Converted time format in crossOverSP to yyyymmdd; get all rows in np2020 that matches the sp,C/P of each date; 
#    iterated through crossOverSP & crossOverAndnp2020 to see if there's a profit using this strategy
# 06-19-2021A: Fixed three bugs: changed 1.01-->1.2 (I lowered it previously to test if the profits work); 
#    induced a sell when option goes <= 50% of what we bought it for
#    modified end of day sale if we didn't sell our option;
# 06-19-2021B: Want to modify the percents for when we buy/sell so we can see if that increases our total profit
# 07-03-2021: Identify the time and price when an option is bought/sold; changed starting time from 9:30AM to 9:35AM; 
#    forgot to use strikeprice for buy/sell loop; fixed rounding from row[5] to row[6] in crossOverSP (corresponds to SP)

In [137]:
# A RSI Crossover occurs when the RSI value turns from a value outside of (30,70) to a value inside of that range 
# (e.g. from 17 to 35 or 88 to 67)

In [138]:
import pandas as pd
import numpy as np
import math
#df = pd.read_csv('Jan 2020 - Dec 2020 merged SPY - Copy.csv')
#df = pd.read_csv('Jan 2020 - Dec 2020 merged SPY.csv')
df = pd.read_csv('RSI-Values-for-SPY.csv')
#df = pd.read_csv('Even-smaller-dataframe.csv')

#npData = df.to_numpy()
npData = df.to_numpy(dtype="O")

In [139]:
df

,Symbol,DateTime,Open,High,Low,Close,PrevClose,Dates,Time,RSI
0,SPY,2/18/2016 4:05,192.68,192.68,192.660,192.6600,192.7800,2/18/2016,4:05:00,NaN
1,SPY,2/18/2016 4:10,192.79,192.87,192.790,192.8700,192.6600,2/18/2016,4:10:00,NaN
2,SPY,2/18/2016 4:15,192.77,192.79,192.770,192.7900,192.8700,2/18/2016,4:15:00,NaN
3,SPY,2/18/2016 4:20,192.90,192.90,192.800,192.8100,192.7900,2/18/2016,4:20:00,NaN
4,SPY,2/18/2016 4:25,192.81,192.88,192.770,192.8800,192.8100,2/18/2016,4:25:00,NaN
...,...,...,...,...,...,...,...,...,...,...
259217,SPY,5/3/2021 15:40,418.37,418.45,418.245,418.3200,418.3790,5/3/2021,15:40:00,37.328493
259218,SPY,5/3/2021 15:45,418.32,418.35,418.140,418.1600,418.3200,5/3/2021,15:45:00,35.744368
259219,SPY,5/3/2021 15:50,418.15,418.25,417.890,417.9381,418.1600,5/3/2021,15:50:00,31.802816
259220,SPY,5/3/2021 15:55,417.94,418.25,417.665,418.1699,417.9381,5/3/2021,15:55:00,27.305693


In [140]:
npData

array([['SPY', '2/18/2016 4:05', 192.68, ..., '2/18/2016', '4:05:00',
        nan],
       ['SPY', '2/18/2016 4:10', 192.79, ..., '2/18/2016', '4:10:00',
        nan],
       ['SPY', '2/18/2016 4:15', 192.77, ..., '2/18/2016', '4:15:00',
        nan],
       ...,
       ['SPY', '5/3/2021 15:50', 418.15, ..., '5/3/2021', '15:50:00',
        31.80281646],
       ['SPY', '5/3/2021 15:55', 417.94, ..., '5/3/2021', '15:55:00',
        27.305692999999998],
       ['SPY', '5/3/2021 16:00', 418.17, ..., '5/3/2021', '16:00:00',
        37.28264286]], dtype=object)

In [141]:
type(npData[0][9])

float

In [142]:
#function for checking if there's an RSI crossover
def isRSICross(prev,cur):
    #if prev <= 30 or prev >= 70:
    #    if 30 < cur < 70:
    #        return True
    if prev <= 30 and cur > 30:
        #up crossover
        return "Up"
    elif prev >= 70 and cur < 70:
        #down crossover
        return "Down"
    return ""

### Found RSI crossovers

In [143]:
crossoverList = []
prevRSI = -1
for row in npData:
    curRSI = row[9]
    if not math.isnan(curRSI):
        if not prevRSI == -1:
            #both curRSI and prevRSI are defined
            typeOfCross = isRSICross(prevRSI,curRSI)
            if typeOfCross != "":
                crossoverList.append((row,typeOfCross))  
                #crossoverList.append((prevRSI, row[9]))
        prevRSI = curRSI

In [144]:
crossoverList[0:10]

[(array(['SPY', '2/18/2016 6:45', 193.69, 193.7, 193.68, 193.7, 193.63,
         '2/18/2016', '6:45:00', 65.51150993], dtype=object), 'Down'),
 (array(['SPY', '2/18/2016 9:55', 192.8, 192.83, 192.61, 192.72, 192.7943,
         '2/18/2016', '9:55:00', 37.74260086], dtype=object), 'Up'),
 (array(['SPY', '2/19/2016 7:00', 191.39, 191.39, 191.34, 191.39, 191.47,
         '2/19/2016', '7:00:00', 32.66788551], dtype=object), 'Up'),
 (array(['SPY', '2/22/2016 5:55', 194.12, 194.15, 194.11, 194.15, 194.15,
         '2/22/2016', '5:55:00', 68.03611045], dtype=object), 'Down'),
 (array(['SPY', '2/22/2016 6:10', 194.22, 194.25, 194.22, 194.22, 194.22,
         '2/22/2016', '6:10:00', 68.63163598], dtype=object), 'Down'),
 (array(['SPY', '2/22/2016 6:35', 194.22, 194.29, 194.22, 194.26, 194.22,
         '2/22/2016', '6:35:00', 62.18564193], dtype=object), 'Down'),
 (array(['SPY', '2/22/2016 10:10', 194.57, 194.7, 194.49, 194.62, 194.575,
         '2/22/2016', '10:10:00', 60.42242438], dtype=object

### Get strikeprice for each row that had RSI-Crossover
We round up if it goes from below 30 to above 30 and say it would be a call ('C'), and round down if it goes down from above 70 to under 70 and say it would be a put ('P')
We also want to only take rows with time b/w 9:30AM and 11:00AM

In [145]:
def beforeTime(time1,time2):
    if len(time1) < len(time2):
        return True
    elif len(time1) > len(time2):
        return False
    else:
        #hrs1,mins1,secs1 = time1.split(":")
        #hrs2,mins2,secs2 = time2.spllit(":")
        return time1.split(":") < time2.split(":")
    
def afterTime(time1,time2):
    if time1 == time2:
        return False
    return not beforeTime(time1,time2)

#test cases
#beforeTime("9:30:00","11:15:00")
#beforeTime("9:30:00","9:45:00")
#beforeTime("16:00:00","14:15:02")
#afterTime("16:00:00","14:15:02")

In [146]:
crossOverSP = [] #date,time,roundedSP,up/down (call/put)
for row,crossType in crossoverList:
    #9:35AM - 11:00AM
    if not beforeTime(row[8],"9:35:00") and not afterTime(row[8],"11:00:00"):
        #modify date from mm/dd/yy to yyyymmdd
        month,day,year = row[7].split("/")
        if len(month) == 1:
            month = "0" + month
        if len(day) == 1:
            day = "0" + day
        dateToAppend = int(year + month + day)
        if crossType == "Up":
            rowToAppend = [dateToAppend,row[8],math.ceil(row[6]),"C"]
        else:
            rowToAppend = [dateToAppend,row[8],math.floor(row[6]),"P"]
        crossOverSP.append(rowToAppend)

In [147]:
crossOverSP[0:10]

[[20160218, '9:55:00', 193, 'C'],
 [20160222, '10:10:00', 194, 'P'],
 [20160223, '10:25:00', 194, 'C'],
 [20160224, '10:00:00', 190, 'C'],
 [20160226, '10:15:00', 196, 'C'],
 [20160226, '10:30:00', 196, 'C'],
 [20160301, '10:50:00', 195, 'P'],
 [20160303, '9:50:00', 199, 'C'],
 [20160303, '10:20:00', 199, 'C'],
 [20160307, '9:55:00', 199, 'P']]

### Use the 2020 dataset with the option prices and get rid of any options that we do not have in the crossOverSP

In [148]:
data2020 = pd.read_csv('2020-mergedData.csv')
np2020 = data2020.to_numpy(dtype="O")

In [149]:
data2020

,Date,TimeBarStart,Ticker,CallPut,Strike,ExpirationDate,OpenTradePrice,HighTradePrice,LowTradePrice,CloseTradePrice,UnderOpenBidPrice,UnderOpenAskPrice,UnderCloseBidPrice,UnderCloseAskPrice,VWAP,Volume,TotalTrades
0,20200103,9:30:00 AM,SPY,P,320.5,20200103,0.57,0.60,0.47,0.47,321.13,321.15,321.55,321.56,0.5865,241,17
1,20200103,9:30:00 AM,SPY,P,305.0,20200103,0.01,0.01,0.01,0.01,321.13,321.16,321.13,321.16,0.0100,6,1
2,20200103,9:30:00 AM,SPY,P,317.0,20200103,0.12,0.12,0.10,0.10,321.18,321.20,321.33,321.34,0.1018,11,2
3,20200103,9:30:00 AM,SPY,P,313.0,20200103,0.02,0.02,0.02,0.02,321.44,321.45,321.44,321.45,0.0200,1,1
4,20200103,9:30:00 AM,SPY,P,321.5,20200103,0.98,0.98,0.82,0.82,321.14,321.16,321.47,321.48,0.9241,279,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885432,20201231,4:14:00 PM,SPY,C,374.0,20201231,0.20,0.25,0.18,0.25,374.01,374.04,374.09,374.10,0.2030,796,38
1885433,20201231,4:14:00 PM,SPY,C,371.0,20201231,3.12,3.12,3.12,3.12,374.09,374.10,374.09,374.10,3.1200,2,1
1885434,20201231,4:14:00 PM,SPY,C,372.0,20201231,2.07,2.10,2.03,2.10,374.02,374.04,374.06,374.09,2.0618,17,5
1885435,20201231,4:14:00 PM,SPY,C,375.0,20201231,0.01,0.01,0.01,0.01,374.07,374.10,374.07,374.10,0.0100,10,1


In [150]:
np2020[0:10]

array([[20200103, '9:30:00 AM', 'SPY', 'P', 320.5, 20200103, 0.57, 0.6,
        0.47, 0.47, 321.13, 321.15, 321.55, 321.56, 0.5865, 241, 17],
       [20200103, '9:30:00 AM', 'SPY', 'P', 305.0, 20200103, 0.01, 0.01,
        0.01, 0.01, 321.13, 321.16, 321.13, 321.16, 0.01, 6, 1],
       [20200103, '9:30:00 AM', 'SPY', 'P', 317.0, 20200103, 0.12, 0.12,
        0.1, 0.1, 321.18, 321.2, 321.33, 321.34, 0.1018, 11, 2],
       [20200103, '9:30:00 AM', 'SPY', 'P', 313.0, 20200103, 0.02, 0.02,
        0.02, 0.02, 321.44, 321.45, 321.44, 321.45, 0.02, 1, 1],
       [20200103, '9:30:00 AM', 'SPY', 'P', 321.5, 20200103, 0.98, 0.98,
        0.82, 0.82, 321.14, 321.16, 321.47, 321.48, 0.9241, 279, 43],
       [20200103, '9:30:00 AM', 'SPY', 'P', 316.5, 20200103, 0.07, 0.08,
        0.07, 0.07, 321.31, 321.33, 321.41, 321.42, 0.0757, 35, 3],
       [20200103, '9:30:00 AM', 'SPY', 'P', 318.5, 20200103, 0.21, 0.21,
        0.21, 0.21, 321.31, 321.32, 321.31, 321.32, 0.21, 30, 1],
       [20200103, '9:

In [151]:
#could've been done using for loop to optimize speed
curdate = -1
crossoverspPointer = 0
np2020Pointer = 0
crossOverAndnp2020 = []
while np2020Pointer < len(np2020) and crossoverspPointer < len(crossOverSP):
    #print(1)
    if curdate != np2020[np2020Pointer][0]:
        #if the dates are not the same in both crossOverSP and np2020, then increment their pointers forward
        curdate = np2020[np2020Pointer][0]
        while crossoverspPointer < len(crossOverSP) and crossOverSP[crossoverspPointer][0] < np2020[np2020Pointer][0]:
            #keep incrementing crossoversp pointer forward until the date matches up
            crossoverspPointer += 1
            #print(2)
            
        while crossoverspPointer < len(crossOverSP) and np2020Pointer < len(np2020) and crossOverSP[crossoverspPointer][0] > np2020[np2020Pointer][0]:
            #crossoversp date is ahead of np2020 date, so move np2020 pointer forward
            np2020Pointer += 1
            #print(3)
        
    else:
        #dates match, so store all of this same date from crossOverSp to a set, then keep iterating through np2020
        crossOverRowsDate = set()
        while crossoverspPointer < len(crossOverSP) and crossOverSP[crossoverspPointer][0] == curdate:
            crossOverRowsDate.add((crossOverSP[crossoverspPointer][2],crossOverSP[crossoverspPointer][3]))
            crossoverspPointer += 1
            #print(4)
        
        #iterate through np2020 to get all instances of (strikeprice,C/P) that matches that in the crossOverRowsDate set
        while np2020Pointer < len(np2020) and np2020[np2020Pointer][0] == curdate:
            if (np2020[np2020Pointer][4],np2020[np2020Pointer][3]) in crossOverRowsDate:
                crossOverAndnp2020.append(np2020[np2020Pointer])
            #print(5)
            np2020Pointer += 1

In [152]:
#use set for each date in crossOverSP, and then iterate through each date in np2020
#make sure the dates are formatted the same
np2020[0]
crossOverAndnp2020[0:10]

[array([20200106, '9:30:00 AM', 'SPY', 'P', 322.0, 20200106, 1.64, 1.79,
        1.45, 1.61, 320.49, 320.5, 320.59, 320.6, 1.6825, 1963, 165],
       dtype=object),
 array([20200106, '9:30:00 AM', 'SPY', 'P', 321.0, 20200106, 0.97, 1.11,
        0.71, 0.96, 320.5, 320.51, 320.54, 320.56, 1.0123, 447, 93],
       dtype=object),
 array([20200106, '9:31:00 AM', 'SPY', 'P', 322.0, 20200106, 1.59, 1.59,
        1.3, 1.3, 320.6, 320.61, 320.92, 320.94, 1.4092, 196, 40],
       dtype=object),
 array([20200106, '9:31:00 AM', 'SPY', 'P', 321.0, 20200106, 0.94, 0.94,
        0.73, 0.73, 320.6, 320.61, 320.94, 320.95, 0.7954, 558, 48],
       dtype=object),
 array([20200106, '9:32:00 AM', 'SPY', 'P', 322.0, 20200106, 1.29, 1.31,
        1.1, 1.15, 320.96, 320.97, 321.13, 321.14, 1.1842, 864, 51],
       dtype=object),
 array([20200106, '9:32:00 AM', 'SPY', 'P', 321.0, 20200106, 0.73, 0.74,
        0.59, 0.63, 320.96, 320.97, 321.16, 321.17, 0.6162, 3754, 156],
       dtype=object),
 array([202001

### Get opentradeprice of the strike price of the first rsi crossover of the day and then multiply that amount by 1.25 and see if this occurs. If it does occur that means we buy the option price and then we see if the option goes down 50% first or goes up 20% 

In [153]:
#convert standard time to military time
#9:31:00 AM --> 9:31:00
#9:31:00 PM --> 21:31:00
def standardToMilitary(time):
    
    if time == "12:00:00 PM":
        return "12:00:00"
    elif time == "12:00:00 AM":
        return "00:00:00"
    
    timeAlone,daynight = time.split(" ")
    hour,minute,second = timeAlone.split(":")
    
    if daynight == "PM":
        if hour == "12":
            return hour + ":" + minute + ":" + second
        return str(int(hour) + 12) + ":" + minute + ":" + second
    if hour == "12":
        hour = "0"
    return hour + ":" + minute + ":" + second
#standardToMilitary("11:00:32 AM")
#standardToMilitary("1:00:32 PM")
#standardToMilitary("12:00:32 PM")
standardToMilitary("12:00:13 AM")

'0:00:13'

In [154]:
#to find the opentradeprice we need the date, time of crossover, SP, and call/put
curdate = -1
firstRSICrossoverOTP = -1
daysProfit, daysLost = 0,0
cPtr = 0
daysAndProfit = [] #(date,profit/loss)
#buyInd, profitSellInd, lossSellInd = 1.1, 1.2, 0.8
buyInd, profitSellInd, lossSellInd = 1.01, 1.5, 0.8
activity = [] # date, time, buy/sell, price, SP, C/P
for row in crossOverSP:
    if curdate == row[0]:
        #we used this date already so skip
        continue
    curdate,crossoverTime,strikeprice,callPut = row[0],row[1],row[2],row[3]
    
    if cPtr >= len(crossOverAndnp2020):
        break
    if curdate < crossOverAndnp2020[cPtr][0]:
        #this crossOverSP date is behind crossOverAndnp2020[cPtr], so we can no longer use it
        continue
    
    bought = False
    soldToday = False
    while cPtr < len(crossOverAndnp2020) and curdate > crossOverAndnp2020[cPtr][0]:
        #keep moving cPtr forward until the date catches up to crossOverSP's date
        cPtr += 1
        #print(1)

    if cPtr < len(crossOverAndnp2020) and curdate == crossOverAndnp2020[cPtr][0]:
        
        prevPrice = None
        
        #keep moving cPtr forward until we reached the point where the strikeprice and callput matches
        while cPtr < len(crossOverAndnp2020) and \
            (strikeprice,callPut) != (crossOverAndnp2020[cPtr][4],crossOverAndnp2020[cPtr][3]):
            #not afterTime(standardToMilitary(crossOverAndnp2020[cPtr][1]),row[1]):
            cPtr += 1
            #print(2)
        
        #found the first crossover of that date in crossOverAndnp2020; get the OTP
        otpCrossoverStart = crossOverAndnp2020[cPtr][6]
        
        while cPtr < len(crossOverAndnp2020) and curdate == crossOverAndnp2020[cPtr][0]:
            #print(3)
            if (strikeprice,callPut) != (crossOverAndnp2020[cPtr][4],crossOverAndnp2020[cPtr][3]):
                cPtr += 1
                #print(4)
                continue
            
            #keep track of prev price so we can use for subtraction if the day is over and we want to sell option
            prevPrice = crossOverAndnp2020[cPtr][6]
            prevTime = crossOverAndnp2020[cPtr][1]
            
            #if we bought something then check the rest of the day to see if there's a +20% or -50%
            #if bought and SP >= 1.2 * boughtPrice:
            if bought and crossOverAndnp2020[cPtr][6] >= profitSellInd * boughtPrice:
                daysProfit += 1
                daysAndProfit.append((curdate,profitSellInd))
                #we sold here
                activity.append([curdate,crossOverAndnp2020[cPtr][1],"Sell",crossOverAndnp2020[cPtr][6],strikeprice,callPut])
                soldToday = True
                bought = False
                #print(5)
                break
            elif bought and crossOverAndnp2020[cPtr][6] <= lossSellInd * boughtPrice:
                #we hit -50%, so sell it
                daysAndProfit.append((curdate,lossSellInd))
                activity.append([curdate,crossOverAndnp2020[cPtr][1],"Sell",crossOverAndnp2020[cPtr][6],strikeprice,callPut])
                daysLost += 1
                soldToday = True
                bought = False
                break
            
            #comeback2 need to implement
            #if not bought, we need to make sure that crossOverAndnp2020's time is between 9:35AM to 11:00AM
            #crossOverAndnp2020's time has to be >= crossOverSP's time
            betweenTimeCheck = False
            while beforeTime(standardToMilitary(crossOverAndnp2020[cPtr][1]),crossoverTime): 
                #crossOverAndnp2020 is standard, crossOverTime is military
                cPtr += 1
                betweenTimeCheck = True
            if betweenTimeCheck:
                continue #reset to do strikeprice C/P check 
            
            if not bought and afterTime(standardToMilitary(crossOverAndnp2020[cPtr][1]),"11:00:00"):
                #if not bought and after 11:00AM then we are past the time we want to buy for this date
                while cPtr < len(crossOverAndnp2020) and crossOverAndnp2020[cPtr][0] == curdate:
                    cPtr += 1
                break
            #if not bought then check if there's a 1.25x increase in OTP
            #if not bought and not soldToday and crossOverAndnp2020[cPtr][6] >= 1.25 * otpCrossoverStart:
            elif not bought and crossOverAndnp2020[cPtr][6] >= buyInd * otpCrossoverStart:
                #found 1.25x increase, make a purchase
                bought = True
                boughtPrice = crossOverAndnp2020[cPtr][6]
                activity.append([curdate,crossOverAndnp2020[cPtr][1],"Buy",boughtPrice,strikeprice,callPut])
                #print(6)
            
            cPtr += 1
        
        if bought:
            #didn't sell it, so sell and incur what the profit/loss was
            if prevPrice:
                #profitPerc = (prevPrice - boughtPrice) / boughtPrice
                #30 --> 100: (100-30)/30
                #100 --> 30: (30-100)/100 = -0.7
                profitPerc = prevPrice / boughtPrice
                #A --> B: B/A percent profit
                daysAndProfit.append((curdate,profitPerc))
                activity.append([curdate,prevTime,"Sell",prevPrice,strikeprice,callPut])

In [155]:
#daysAndProfit
#(indicator for buy,indicator for sell,indicator for when loss is too much) : totalprofit
#(1.25,1.2,0.5): 0.10733438842491431
#(1.25,1.2,0.8): 0.6475468210338499
#(1.1,1.2,0.8): 0.008999755843624115

In [156]:
#print(daysProfit,daysLost)
#totalProfit = (1.2)**daysProfit*(0.5)**daysLost
#print(totalProfit)
totalProfit = 1
#totalProfit = totalProfit *= x[1] for x in daysAndProfit
for x in daysAndProfit:
    totalProfit *= x[1]
totalProfit

0.024903104499507923

In [157]:
activity[0:10]

[[20200113, '9:45:00 AM', 'Buy', 0.26, 327, 'C'],
 [20200113, '9:52:00 AM', 'Sell', 0.2, 327, 'C'],
 [20200212, '9:52:00 AM', 'Buy', 0.3, 336, 'P'],
 [20200212, '9:58:00 AM', 'Sell', 0.24, 336, 'P'],
 [20200224, '9:35:00 AM', 'Buy', 1.23, 325, 'C'],
 [20200224, '10:29:00 AM', 'Sell', 0.96, 325, 'C'],
 [20200302, '9:35:00 AM', 'Buy', 2.96, 297, 'P'],
 [20200302, '10:39:00 AM', 'Sell', 2.34, 297, 'P'],
 [20200320, '10:32:00 AM', 'Buy', 4.62, 240, 'C'],
 [20200320, '10:47:00 AM', 'Sell', 3.62, 240, 'C']]

In [158]:
crossOverSP[0:10]

[[20160218, '9:55:00', 193, 'C'],
 [20160222, '10:10:00', 194, 'P'],
 [20160223, '10:25:00', 194, 'C'],
 [20160224, '10:00:00', 190, 'C'],
 [20160226, '10:15:00', 196, 'C'],
 [20160226, '10:30:00', 196, 'C'],
 [20160301, '10:50:00', 195, 'P'],
 [20160303, '9:50:00', 199, 'C'],
 [20160303, '10:20:00', 199, 'C'],
 [20160307, '9:55:00', 199, 'P']]

In [159]:
crossOverAndnp2020[0:10]

[array([20200106, '9:30:00 AM', 'SPY', 'P', 322.0, 20200106, 1.64, 1.79,
        1.45, 1.61, 320.49, 320.5, 320.59, 320.6, 1.6825, 1963, 165],
       dtype=object),
 array([20200106, '9:30:00 AM', 'SPY', 'P', 321.0, 20200106, 0.97, 1.11,
        0.71, 0.96, 320.5, 320.51, 320.54, 320.56, 1.0123, 447, 93],
       dtype=object),
 array([20200106, '9:31:00 AM', 'SPY', 'P', 322.0, 20200106, 1.59, 1.59,
        1.3, 1.3, 320.6, 320.61, 320.92, 320.94, 1.4092, 196, 40],
       dtype=object),
 array([20200106, '9:31:00 AM', 'SPY', 'P', 321.0, 20200106, 0.94, 0.94,
        0.73, 0.73, 320.6, 320.61, 320.94, 320.95, 0.7954, 558, 48],
       dtype=object),
 array([20200106, '9:32:00 AM', 'SPY', 'P', 322.0, 20200106, 1.29, 1.31,
        1.1, 1.15, 320.96, 320.97, 321.13, 321.14, 1.1842, 864, 51],
       dtype=object),
 array([20200106, '9:32:00 AM', 'SPY', 'P', 321.0, 20200106, 0.73, 0.74,
        0.59, 0.63, 320.96, 320.97, 321.16, 321.17, 0.6162, 3754, 156],
       dtype=object),
 array([202001

In [160]:
#for row in crossOverAndnp2020:
#    if row[0] == 20200124 and row[3] == "C":
#        print(row)

In [161]:
crossOverAndnp2020[0]

array([20200106, '9:30:00 AM', 'SPY', 'P', 322.0, 20200106, 1.64, 1.79,
       1.45, 1.61, 320.49, 320.5, 320.59, 320.6, 1.6825, 1963, 165],
      dtype=object)